In [560]:
import pandas as pd
import numpy as np
import locale
import ast
import datetime
from openpyxl import Workbook,load_workbook,utils
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
import numpy as np
from openpyxl.styles import PatternFill, Border, Side
from openpyxl.formatting.rule import FormulaRule

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')
green_fill = PatternFill(start_color='FF00FF00', end_color='FF00FF00', fill_type='solid')
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

In [561]:
Months = {"Jan":1,
         "Feb":2,
         "Mar":3,
         "Apr":4,
         "May":5,
         "Jun":6,
         "Jul":7,
         "Aug":8,
         "Sep":9,
         "Oct":10,
         "Nov":11,
         "Dec":12}

In [562]:
def unwrap(val):
    # peel back multiple string nestings until it's a real object
    while isinstance(val, str):
        try:
            val = ast.literal_eval(val)
        except Exception:
            break
    return val

In [563]:
#Master Spreadsheet
master = pd.read_csv("25KChallenge.csv",converters={'Trade Exit': unwrap})

In [564]:
#Rename Master spreadsheet's column names
Col_Names = [i for i in master.iloc[0,:]]
master.columns = Col_Names
master['Index'] = master.index+1
master

,Trade #,Ticker,Trade Enter,Trade Exit,Exp Date,Strike,C/P,Initial Contracts,# of Contracts,Average Cost/Option,$ Average Cost,Market Value,% Gain,$ Gain,Open/Closed,Notes,Index
0,Trade #,Ticker,Trade Enter,Trade Exit,Exp Date,Strike,C/P,Initial Contracts,# of Contracts,Average Cost/Option,$ Average Cost,Market Value,% Gain,$ Gain,Open/Closed,Notes,1
1,1,AMZN,16-May,3-Jun,16-Jan,195,C,1,1,"$2,892.50","$2,892.50","$2,970.00",2.68%,$77.50,Closed,NaN,2
2,2,ANF,16-May,28-May,16-Jan,70,C,1,1,"$2,140.00","$2,140.00","$4,000.00",86.92%,"$1,860.00",Closed,NaN,3
3,3,META,16-May,19-May,15-Aug,640,C,1,1,"$4,250.00","$4,250.00","$4,900.00",15.29%,$650.00,Closed,NaN,4
4,4,QQQ,16-May,19-May,21-May,522,P,2,2,$495.00,$990.00,"$1,280.00",29.29%,$290.00,Closed,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,286,META,13-Aug,14-Aug,29-Aug,770,P,1,1,"$1,070.00","$1,070.00",$925.00,-13.55%,($145.00),closed,NaN,288
288,287,IWM,13-Aug,14-Aug,17-Oct,228,C,3,3,"$1,031.00","$3,093.00","$2,505.00",-19.01%,($588.00),closed,NaN,289
289,288,IWM,13-Aug,14-Aug,29-Aug,231,P,2,2,$372.00,$744.00,"$1,090.00",46.51%,$346.00,closed,NaN,290
290,289,QQQ,13-Aug,14-Aug,19-Sep,585,P,2,2,"$1,215.00","$2,430.00","$2,430.00",0.00%,$0.00,closed,NaN,291


In [565]:
#All trade entrance dates, Tickers, and C/Ps in Master spreadsheet
Dates = master.loc[1:,['Index','Ticker','Trade Enter', 'Strike','C/P','Exp Date', "Initial Contracts", 'Trade Exit', '$ Gain']]
Dates['Strike'] = [locale.currency(float(price)) for price in Dates['Strike']]
Dates

,Index,Ticker,Trade Enter,Strike,C/P,Exp Date,Initial Contracts,Trade Exit,$ Gain
1,2,AMZN,16-May,$195.00,C,16-Jan,1,3-Jun,$77.50
2,3,ANF,16-May,$70.00,C,16-Jan,1,28-May,"$1,860.00"
3,4,META,16-May,$640.00,C,15-Aug,1,19-May,$650.00
4,5,QQQ,16-May,$522.00,P,21-May,2,19-May,$290.00
5,6,MSFT,16-May,$500.00,C,16-Jan,1,26-Jun,"$1,289.00"
...,...,...,...,...,...,...,...,...,...
287,288,META,13-Aug,$770.00,P,29-Aug,1,14-Aug,($145.00)
288,289,IWM,13-Aug,$228.00,C,17-Oct,3,14-Aug,($588.00)
289,290,IWM,13-Aug,$231.00,P,29-Aug,2,14-Aug,$346.00
290,291,QQQ,13-Aug,$585.00,P,19-Sep,2,14-Aug,$0.00


In [566]:
def clean_trade_exit(val):
    if pd.isna(val) or val == 'nan' or val == 'NaN':
        return 'NaN'
    try:
        # If it's a string representation of a list or nested structure
        if isinstance(val, str) and ('[' in val or '{' in val):
            val = unwrap(val)
        
        # If it's a list, take the first element
        if isinstance(val, (list, tuple)):
            val = val[0]
            
        return str(val)
    except Exception:
        return 'NaN'

# Clean the Trade Exit column
Dates['Trade Exit'] = Dates['Trade Exit'].apply(clean_trade_exit)
# print("Sample of cleaned Trade Exit dates:")
# print(Dates['Trade Exit'].head())

In [567]:
# Process Trade Enter dates
TRADE_ENTRANCE_DATES = [datetime.date(year=2025,month=Months[y[1]], day=int(y[0])) for date in list(Dates.loc[:,'Trade Enter']) if (y:=str(date).split('-'))]

# Process Trade Exit dates with better handling
TRADE_EXIT_DATES = []
for date in Dates['Trade Exit']:
    if date == 'NaN':
        TRADE_EXIT_DATES.append('NaN')
    else:
        parts = str(date).split('-')
        if len(parts) > 1:  # Only append if we have a valid date format
            TRADE_EXIT_DATES.append(parts)
        else:
            TRADE_EXIT_DATES.append('NaN')

# Update the dataframe
Dates['Trade Enter'] = TRADE_ENTRANCE_DATES
Dates['Trade Exit'] = [datetime.date(year=2025,month=Months[date[1]], day=int(date[0])).strftime("%b %d, %Y") if isinstance(date,list) else 'NaN' for date in TRADE_EXIT_DATES]
Dates

,Index,Ticker,Trade Enter,Strike,C/P,Exp Date,Initial Contracts,Trade Exit,$ Gain
1,2,AMZN,2025-05-16,$195.00,C,16-Jan,1,"Jun 03, 2025",$77.50
2,3,ANF,2025-05-16,$70.00,C,16-Jan,1,"May 28, 2025","$1,860.00"
3,4,META,2025-05-16,$640.00,C,15-Aug,1,"May 19, 2025",$650.00
4,5,QQQ,2025-05-16,$522.00,P,21-May,2,"May 19, 2025",$290.00
5,6,MSFT,2025-05-16,$500.00,C,16-Jan,1,"Jun 26, 2025","$1,289.00"
...,...,...,...,...,...,...,...,...,...
287,288,META,2025-08-13,$770.00,P,29-Aug,1,"Aug 14, 2025",($145.00)
288,289,IWM,2025-08-13,$228.00,C,17-Oct,3,"Aug 14, 2025",($588.00)
289,290,IWM,2025-08-13,$231.00,P,29-Aug,2,"Aug 14, 2025",$346.00
290,291,QQQ,2025-08-13,$585.00,P,19-Sep,2,"Aug 14, 2025",$0.00


In [568]:
type(list(Dates.loc[:,'Trade Enter'])[0])

datetime.date

In [569]:
TRADE_EXP_DATE = [datetime.date(year=2025, month=Months[y[1]], day=int(y[0])).strftime("%b %d, %Y") if (Months[y[1]]> Months[trade_enter.strftime('%b')] or (Months[y[1]] == Months[trade_enter.strftime('%b')] and int(y[0]) > int(trade_enter.strftime("%d")))) else datetime.date(year=2026, month=Months[y[1]], day=int(y[0])).strftime("%b %d, %Y") for exp_date,trade_enter in zip(list(Dates.loc[:,'Exp Date']), list(Dates.loc[:,'Trade Enter'])) if (y:=str(exp_date).split('-'))]
Dates['Exp Date'] = TRADE_EXP_DATE
Dates['Trade Enter'] = [date.strftime("%b %d, %Y") for date in Dates['Trade Enter']]
Dates

,Index,Ticker,Trade Enter,Strike,C/P,Exp Date,Initial Contracts,Trade Exit,$ Gain
1,2,AMZN,"May 16, 2025",$195.00,C,"Jan 16, 2026",1,"Jun 03, 2025",$77.50
2,3,ANF,"May 16, 2025",$70.00,C,"Jan 16, 2026",1,"May 28, 2025","$1,860.00"
3,4,META,"May 16, 2025",$640.00,C,"Aug 15, 2025",1,"May 19, 2025",$650.00
4,5,QQQ,"May 16, 2025",$522.00,P,"May 21, 2025",2,"May 19, 2025",$290.00
5,6,MSFT,"May 16, 2025",$500.00,C,"Jan 16, 2026",1,"Jun 26, 2025","$1,289.00"
...,...,...,...,...,...,...,...,...,...
287,288,META,"Aug 13, 2025",$770.00,P,"Aug 29, 2025",1,"Aug 14, 2025",($145.00)
288,289,IWM,"Aug 13, 2025",$228.00,C,"Oct 17, 2025",3,"Aug 14, 2025",($588.00)
289,290,IWM,"Aug 13, 2025",$231.00,P,"Aug 29, 2025",2,"Aug 14, 2025",$346.00
290,291,QQQ,"Aug 13, 2025",$585.00,P,"Sep 19, 2025",2,"Aug 14, 2025",$0.00


In [570]:
#Tradier_Activities Spreadsheet
#Deleting rows whose amount is 0, or whose absolute value is greater than 50
Active = pd.read_csv("Tradier_activities_2025-05-11_2025-08-14.csv")
Active['Index'] = Active.index+1
Active = Active[Active['Amount']!= 0]
#Active = Active[abs(Active['Amount'])<=50]
Active;

In [571]:
#Extracting ticker names from symbols in Tradier_activities
Symbols = [str(k) for k in list(Active.loc[:,'Symbol'])]
for i in range(len(Symbols)):
    Symbol = Symbols[i]
    for l in range(len(Symbol)):
        if(Symbol[l].isdigit()):
            Symbols[i] = Symbol[0:l].strip()
            break

In [572]:
Descriptions = [str(k).split() for k in list(Active.loc[:,'Description'])]
tickers = [row[0] for row in Descriptions]
Strike_prices = [''.join(desc[4:5]) for desc in Descriptions]
Exp_Date = [' '.join(row[1:4]) for row in Descriptions]

In [573]:
#Types
Types = Active.loc[:,['Symbol','Date','Quantity','Index','Amount']]
Types

,Symbol,Date,Quantity,Index,Amount
0,TSLA260116C00320000,"Aug 22, 2025",-6,1,32579.32
1,TSLA250919P00300000,"Aug 22, 2025",-1,2,409.88
2,PLTR250829P00157500,"Aug 22, 2025",-1,3,256.88
3,BMNR250919P00046000,"Aug 22, 2025",-1,4,374.88
4,PDD251017C00110000,"Aug 22, 2025",-1,5,1944.88
...,...,...,...,...,...
2395,MSFT260116C00500000,"May 16, 2025",2,2396,-3944.94
2396,META250815C00640000,"May 16, 2025",2,2397,-8630.94
2397,QQQ250521P00522000,"May 16, 2025",4,2398,-1969.86
2398,MSFT250815C00450000,"May 15, 2025",2,2399,-5400.94


In [574]:
#Call or Put
Descriptions = [str(k) for k in list(Active.loc[:,'Description'])]
CoP = ["C" if 'Call' in Desc else ("P" if "Put" in Desc else "N/A") for Desc in Descriptions]
Types["C/P"] = CoP
Types["Ticker"] = Symbols
Types;

In [575]:
#Rename the columns to match master spreadsheet columns names
Types = Types.rename(columns={'Date':'Trade Enter', "Quantity":'Initial Contracts'})
Types['Exp Date'] = Exp_Date
Types['Strike'] = Strike_prices
Types['Strike'] = Types['Strike'].apply(lambda x: x.replace(",",""))
Types['Amount'] = Types['Amount'].apply(lambda x: float(x))
Types = Types.reindex(columns=['Index','Ticker','Trade Enter', 'Strike','C/P','Exp Date', "Initial Contracts",'Trade Exit', "$ Gain",'Amount','Symbol'])
Types

,Index,Ticker,Trade Enter,Strike,C/P,Exp Date,Initial Contracts,Trade Exit,$ Gain,Amount,Symbol
0,1,TSLA,"Aug 22, 2025",$320.00,C,"Jan 16, 2026",-6,NaN,NaN,32579.32,TSLA260116C00320000
1,2,TSLA,"Aug 22, 2025",$300.00,P,"Sep 19, 2025",-1,NaN,NaN,409.88,TSLA250919P00300000
2,3,PLTR,"Aug 22, 2025",$157.50,P,"Aug 29, 2025",-1,NaN,NaN,256.88,PLTR250829P00157500
3,4,BMNR,"Aug 22, 2025",$46.00,P,"Sep 19, 2025",-1,NaN,NaN,374.88,BMNR250919P00046000
4,5,PDD,"Aug 22, 2025",$110.00,C,"Oct 17, 2025",-1,NaN,NaN,1944.88,PDD251017C00110000
...,...,...,...,...,...,...,...,...,...,...,...
2395,2396,MSFT,"May 16, 2025",$500.00,C,"Jan 16, 2026",2,NaN,NaN,-3944.94,MSFT260116C00500000
2396,2397,META,"May 16, 2025",$640.00,C,"Aug 15, 2025",2,NaN,NaN,-8630.94,META250815C00640000
2397,2398,QQQ,"May 16, 2025",$522.00,P,"May 21, 2025",4,NaN,NaN,-1969.86,QQQ250521P00522000
2398,2399,MSFT,"May 15, 2025",$450.00,C,"Aug 15, 2025",2,NaN,NaN,-5400.94,MSFT250815C00450000


In [576]:
Dates

,Index,Ticker,Trade Enter,Strike,C/P,Exp Date,Initial Contracts,Trade Exit,$ Gain
1,2,AMZN,"May 16, 2025",$195.00,C,"Jan 16, 2026",1,"Jun 03, 2025",$77.50
2,3,ANF,"May 16, 2025",$70.00,C,"Jan 16, 2026",1,"May 28, 2025","$1,860.00"
3,4,META,"May 16, 2025",$640.00,C,"Aug 15, 2025",1,"May 19, 2025",$650.00
4,5,QQQ,"May 16, 2025",$522.00,P,"May 21, 2025",2,"May 19, 2025",$290.00
5,6,MSFT,"May 16, 2025",$500.00,C,"Jan 16, 2026",1,"Jun 26, 2025","$1,289.00"
...,...,...,...,...,...,...,...,...,...
287,288,META,"Aug 13, 2025",$770.00,P,"Aug 29, 2025",1,"Aug 14, 2025",($145.00)
288,289,IWM,"Aug 13, 2025",$228.00,C,"Oct 17, 2025",3,"Aug 14, 2025",($588.00)
289,290,IWM,"Aug 13, 2025",$231.00,P,"Aug 29, 2025",2,"Aug 14, 2025",$346.00
290,291,QQQ,"Aug 13, 2025",$585.00,P,"Sep 19, 2025",2,"Aug 14, 2025",$0.00


In [577]:
Dates = pd.concat([Dates,pd.read_csv('Mahe-Challenge.csv')], ignore_index=True)
Dates

,Index,Ticker,Trade Enter,Strike,C/P,Exp Date,Initial Contracts,Trade Exit,$ Gain
0,2,AMZN,"May 16, 2025",$195.00,C,"Jan 16, 2026",1,"Jun 03, 2025",$77.50
1,3,ANF,"May 16, 2025",$70.00,C,"Jan 16, 2026",1,"May 28, 2025","$1,860.00"
2,4,META,"May 16, 2025",$640.00,C,"Aug 15, 2025",1,"May 19, 2025",$650.00
3,5,QQQ,"May 16, 2025",$522.00,P,"May 21, 2025",2,"May 19, 2025",$290.00
4,6,MSFT,"May 16, 2025",$500.00,C,"Jan 16, 2026",1,"Jun 26, 2025","$1,289.00"
...,...,...,...,...,...,...,...,...,...
329,39,QQQ,"Aug 20, 2025",$564.00,P,"Aug 25, 2025",2,"Aug 21, 2025",$290.00
330,40,PLTR,"Aug 20, 2025",$157.50,P,"Aug 29, 2025",1,"Aug 22, 2025",($395.00)
331,41,CVNA,"Aug 21, 2025",$350.00,P,"Sep 19, 2025",1,"Aug 22, 2025",($820.00)
332,42,QQQ,"Aug 21, 2025",$563.00,P,"Aug 29, 2025",1,"Aug 22, 2025",($420.00)


In [578]:
Dates['$ Gain'].iloc[0]

'$77.50 '

In [579]:
Types['Trade Exit'] = [Types.iloc[k]['Trade Enter'] if Types.iloc[k]['Initial Contracts']<0 else 'NaN' for k in range(Types.shape[0])]
Types;

In [580]:
#All unique tickers in the master spreadsheet
All_Tickers = Dates['Ticker'].unique()
len(All_Tickers)
All_Tickers

array(['AMZN', 'ANF', 'META', 'QQQ', 'MSFT', 'TSLA', 'HOOD', 'AMD',
       'SHOP', 'SOFI', 'ARM', 'BA', 'NVDA', 'COIN', 'AAPL', 'CRWV',
       'RIVN', 'SLV', 'BABA', 'GLD', 'COST', 'CVNA', 'PANW', 'SMCI',
       'CRCL', 'PLTR', 'ASTS', 'APP', 'TTD', 'SPY', 'GOOG', 'OKLO', 'UNH',
       'NFLX', 'SE', 'PCOR', 'LLY', 'QBTS', 'AMC', 'MMM', 'JD', 'SG',
       'DKNG', 'TGT', 'RDDT', 'DOCU', 'LUNR', 'GOOGL', 'WMT', 'IOT', 'MU',
       'NTNX', 'SNOW', 'HIMS', 'AVGO', 'FSLR', 'PDD', 'IWM', 'ZM', 'RIOT',
       'RKLB', 'UBER', 'BMNR', 'BRKB', 'OSCR', 'ZS'], dtype=object)

In [581]:
#Master Spreadsheet dictionary (key=ticker, value =Dataframe of all rows with ticker)
Master_Dict = {x: pd.DataFrame(Dates.loc[Dates['Ticker'] == x]) for x in All_Tickers}
Master_Dict;

In [582]:
#All tickers in the Tradier_activities spreadhseet
All_Active_Tickers = Types['Ticker'].unique()
All_Active_Tickers;

In [583]:
#Tradier_Activities Spreadsheet dictionary (key=ticker, value =Dataframe of all rows with ticker)
Tradier_Activities_Dict = {t: pd.DataFrame(Types.loc[Types['Ticker'] == t]) for t in All_Active_Tickers}
Tradier_Activities_Dict;

In [584]:
def trade_comparisons(ticker):
    features = ['Trade Enter','Strike','C/P','Exp Date']
    trade_table = Tradier_Activities_Dict[ticker]
    master_table = Master_Dict[ticker]
    new_dfs = []

    for index, entry in master_table.iterrows():
        features_matches = trade_table.loc[
            (trade_table[features] == entry[features]).all(axis=1)
        ]
        
        if pd.notna(entry['Trade Exit']):
            exit_match = trade_table.loc[
                    (trade_table['Trade Enter'] == entry['Trade Exit']) &
                    (trade_table['Strike'] == entry['Strike']) &
                    (trade_table['C/P'] == entry['C/P']) &
                    (trade_table['Exp Date'] == entry['Exp Date'])
                ]
        else:
            exit_match = pd.DataFrame()
            
        if not features_matches.empty or not exit_match.empty:
            temp_df = pd.concat([features_matches, exit_match], ignore_index=True)
        else:
            temp_df = pd.DataFrame()

        new_dfs.append([entry, temp_df])
        
    return new_dfs

In [585]:
#Comparison algorithm
table = Tradier_Activities_Dict['AMZN']
series = Master_Dict['AMZN'][['Trade Enter','Strike','C/P','Exp Date']].iloc[0]

In [586]:
#Step 1: Find rows with equal columns: 'Trade Enter','Strike','C/P','Exp Date' 
list_of_dfs=[Tradier_Activities_Dict['AMZN'].iloc[k] for k in range(Tradier_Activities_Dict['AMZN'].shape[0]) if series.equals(Tradier_Activities_Dict['AMZN'][['Trade Enter','Strike','C/P','Exp Date']].iloc[k])]

list_of_dfs[0].to_frame().T;

In [587]:
#Step 2: Find rows with trade exit trade entrance date
# series2 = Master_Dict['AMZN'].iloc[0]['Trade Exit']
# list_of_dfs2=[Tradier_Activities_Dict['AMZN'].iloc[k] for k in range(Tradier_Activities_Dict['AMZN'].shape[0]) if series2 == Tradier_Activities_Dict['AMZN'].iloc[k]['Trade Enter']]
# list_of_dfs2

In [588]:
#sheet = workbook['AMZN']
def add_toExcel(ticker,df1,df2,new_dfs,ws):
    # Keeping track of current row in the excel sheet
    current_row = 0
    
    #Add the first dataframe to the excel sheet while incrementing the current row
    for row in dataframe_to_rows(df1,index=False):
        ws.append(row)
        current_row+=1

    # Add two blank rows
    for _ in range(2):
        ws.append([""] * df1.shape[1])  # Add a blank row with the same number of columns
        current_row += 1
    start_row = current_row+1
    # ws[f'J{start_row}'].value = 'Calc Gain'
    temp_row = start_row+1
    
    #Add the second dataframe to the excel sheet
    for row in dataframe_to_rows(df2,index=False):
        ws.append(row)
        current_row+=1
    
    #Calculate the sum of the column values in "Gain $" in the second dataframe    
    Gain_Sum = 0
    for num in df2['$ Gain']:
        if(num[0] == '('):
            Gain_Sum+=float(num[2:-1].replace(",",""))*-1
        else:
            Gain_Sum+=float(num[1:].replace(",",""))
    ws["I"+str(current_row+1)].value = Gain_Sum
    ws["I"+str(current_row+1)].fill = yellow_fill
    current_row+=1
    
    # Add two blank rows
    for _ in range(2):
        ws.append([""] * df1.shape[1])  # Add a blank row with the same number of columns
        current_row += 1
    
    total_sum=0
    for frame in new_dfs:
        for r in dataframe_to_rows(frame[0].to_frame().T, index=False):
            ws.append(r)
            current_row+=1

        #Add two blank rows
        ws.append([""] * df1.shape[1])
        ws.append([""] * df1.shape[1])
        current_row+=2
        
        if not frame[1].empty:
            if 'Initial Contracts' in frame[1].columns:
                if int(frame[1]['Initial Contracts'].sum()) != 0:
                    frame[1] = Tradier_Activities_Dict[ticker].loc[Tradier_Activities_Dict[ticker]['Symbol'] == frame[1]['Symbol'].iloc[0]]
        
            for f in dataframe_to_rows(frame[1], index=False):
                ws.append(f)
                current_row+=1
                    
            ws.append([""] * frame[1].shape[1])
            current_row+=1
        
            contracts = 0
            #Contracts Sum
            if 'Initial Contracts' in frame[1].columns:
                for i in range(len(frame[1])):
                    if(frame[1].iloc[i]['Initial Contracts'] > 0):
                        contracts+= frame[1].iloc[i]['Initial Contracts']
                # ws["G"+str(current_row)] = frame[1]['# of Contracts'].sum()
                if len(frame[1]) > 0 and current_row - len(frame[1])-1 <= current_row - 1:
                    ws["G"+str(current_row)].value = f'=SUM(G{current_row-len(frame[1])-1}:G{current_row-1})'
                    ws["G"+str(current_row)].fill = yellow_fill
                else:
                    ws["G"+str(current_row)].fill = yellow_fill
                    
                
            #Amount Sum
            if 'Amount' in frame[1].columns:
                # ws["J"+str(current_row)] = locale.currency(float(frame[1]['Amount'].sum()))
                if len(frame[1]) > 0 and current_row - len(frame[1])-1 <= current_row - 1:
                    ws["J"+str(current_row)].value = f'=SUM(J{current_row-len(frame[1])-1}:J{current_row-1})'
                    ws["J"+str(current_row)].fill = yellow_fill
                else:
                    ws["J"+str(current_row)].fill = yellow_fill
            
            # workbook = load_workbook('TradingMasterData.xlsx', data_only=True)
            # ws = workbook[ticker]
            #frame[1]['# of Contracts'].sum()
            
            # if(ws[f'G{current_row}'].value == 0):
            #     try:
                    # ws.cell(row=current_row-len(frame[1]), column=frame[1].shape[1]+1).value = locale.currency(float(frame[1]['Amount'].sum())/contracts)
            ws.cell(row=current_row-len(frame[1])-2, column=frame[1].shape[1]+1).fill = green_fill
                    # ws[f"L{current_row-len(frame[1])}"] = f"=IF(B1=0, \"Error\", J{current_row}/{contracts})"
            ws[f"L{current_row-len(frame[1])-2}"].value = f"=IF({"G"+str(current_row)}=0, ROUND(SUM(J{current_row-len(frame[1])-1}:J{current_row-1})/{contracts}, 2), "")"
            ws[f"J{temp_row}"].value = ws[f"J{current_row}"].value
            ws[f'K{temp_row}'].value = f'=L{current_row-len(frame[1])-2}*{frame[0]['Initial Contracts']}'
            temp_row+=1
        else:
            temp_row+=1
            
                
        for _ in range(3):
            ws.append([""] * df1.shape[1])  # Add a blank row with the same number of columns
            current_row += 1
        
    ws.cell(row=ws.max_row, column=ws.max_column-1).value = "Total:"
    # ws.cell(row=ws.max_row, column=ws.max_column).value = locale.currency(total_sum)
    ws.cell(row=ws.max_row, column=ws.max_column).fill = green_fill
    
    ws[f"L{ws.max_row}"] = f'=SUM(L1:L{ws.max_row-1})'
    ws[f'K{temp_row}'].fill = yellow_fill
    ws[f'K{temp_row}'].value = f'=ROUND(SUM(K{start_row+1}:K{temp_row-1}),2)'
    ws[f'K{start_row}'].value = 'Calculated $ Gain/25k share'
    ws[f'J{temp_row}'].fill = yellow_fill
    ws[f'J{temp_row}'].value = f'=ROUND(SUM(J{start_row+1}:J{temp_row-1}),2)'
    ws[f'J{start_row}'].value = 'Total Gain'
    
    return [Gain_Sum, f"L{ws.max_row}", f'K{temp_row}', f'J{temp_row}']

In [589]:
ticker_gain_profit = []

In [590]:
filename = 'TradingMasterData.xlsx'

#Loads the excel file temporarily in the memory
#workbook represents the entire excel sheet
workbook = Workbook()
ws = workbook.active
ws.title = 'Ticker Info'
# if "AMZN" in workbook.sheetnames:  # If sheet exists, delete it
#     del workbook['AMZN']
for ticker in All_Tickers:
    if(ticker in All_Active_Tickers):
        workbook.create_sheet(title=ticker)
        ws = workbook[ticker]
        sorted_indices = pd.to_datetime(Tradier_Activities_Dict[ticker]['Trade Enter'], format='%b %d, %Y').sort_values(ascending=False).argsort()
        Tradier_Activities_Dict[ticker] = Tradier_Activities_Dict[ticker].iloc[sorted_indices].reset_index(drop=True)
        Master_Gain, Trader_Gain_loc, Calc_Gain_loc, Total_Gain_loc = add_toExcel(ticker,Tradier_Activities_Dict[ticker], Master_Dict[ticker], trade_comparisons(ticker), ws)
        #print(Trader_Gain_loc)
        ticker_gain_profit.append([ticker, Master_Gain, Trader_Gain_loc, Calc_Gain_loc,Total_Gain_loc])
    else:
        print(f"{ticker} not found in active trade info")
        workbook.create_sheet(title=ticker)
        ws = workbook[ticker]
        
        current_row = 0
        for row in dataframe_to_rows(Master_Dict[ticker],index=False):
            ws.append(row)
            current_row+=1

        # Add two blank rows
        # for _ in range(2):
        #     ws.append([""] * df1.shape[1])  # Add a blank row with the same number of columns
        #     current_row += 1
        
        Gain_Sum = 0
        for num in Master_Dict[ticker]['$ Gain']:
            if(num[0] == '('):
                Gain_Sum+=float(num[2:-1].replace(",",""))*-1
            else:
                Gain_Sum+=float(num[1:].replace(",",""))
        ws["I"+str(current_row+1)] = Gain_Sum
        ws["I"+str(current_row+1)].fill = yellow_fill
        ticker_gain_profit.append([ticker, Gain_Sum, "", "", ""])
        
        
        
ws=workbook['Ticker Info']
# df_ticker_gain_profit = pd.DataFrame(ticker_gain_profit,columns=['Ticker','Master Gain', 'Trader Gain','Master Result', 'Trader Result'])
# header = dataframe_to_rows(df_ticker_gain_profit, index=False)[0]
# ws.append(['Ticker','Master Gain', 'Trader Gain','Calc Gain','Master Result', 'Trader Result'])
ws.append(['Ticker','Master Gain','Total Gain', 'Calc Gain'])

for t in range(0,len(ticker_gain_profit)):
    ws[f'A{t+2}'].value = f'{ticker_gain_profit[t][0]}'
    ws[f'B{t+2}'].value = ticker_gain_profit[t][1]
    # if ticker_gain_profit[t][2]:
    #     ws[f'C{t+2}'].value = f"={utils.quote_sheetname(ticker_gain_profit[t][0])}!{ticker_gain_profit[t][2]}"
    # else:
    #     ws[f'C{t+2}'].value = 0
    if ticker_gain_profit[t][4]:
        ws[f'C{t+2}'].value = f"={utils.quote_sheetname(ticker_gain_profit[t][0])}!{ticker_gain_profit[t][4]}"
    else:
        ws[f'C{t+2}'].value = 0
    
    if ticker_gain_profit[t][3]:
        ws[f'D{t+2}'].value = f"={utils.quote_sheetname(ticker_gain_profit[t][0])}!{ticker_gain_profit[t][3]}"
    else:
        ws[f'D{t+2}'].value = 0
    # ws.cell(row=t+2, column=5).value = f"=IFS(B{t+2}<0, \"Loss\", B{t+2}>0,\"Gain\",B{t+2}=0,\"\")"
    # ws.cell(row=t+2, column=6).value = f"=IFS(C{t+2}<0, \"Loss\", C{t+2}>0, \"Gain\",C{t+2}=0, \"\")"
    
#Normalize "Master Gain" and "Trader Gain" columns
# for i in range(len(df_ticker_gain_profit['Master Gain'])):
#     df_ticker_gain_profit['Master Gain'].iloc[i] = float(df_ticker_gain_profit['Master Gain'].iloc[i].replace("$",""))
#     df_ticker_gain_profit['Trader Gain'].iloc[i] = float(df_ticker_gain_profit['Trader Gain'].iloc[i].replace("$",""))


# ws.cell(row=ws.max_row+1, column=2).value = locale.currency(df_ticker_gain_profit['Master Gain'].sum())
# ws.cell(row=ws.max_row, column=3).value = locale.currency(df_ticker_gain_profit['Trader Gain'].sum())
ws.cell(row=ws.max_row+1, column=2).fill = green_fill
ws.cell(row=ws.max_row, column=3).fill = green_fill
ws.cell(row=ws.max_row, column=4).fill = green_fill
# ws.cell(row=ws.max_row, column=5).fill = green_fill
# ws.cell(row=ws.max_row, column=6).fill = green_fill

ws[f"B{ws.max_row}"].value = f'=SUM(B{ws.max_row-len(ticker_gain_profit)}:B{ws.max_row-1})'
ws[f"C{ws.max_row}"].value = f'=SUM(C{ws.max_row-len(ticker_gain_profit)}:C{ws.max_row-1})'
ws[f"D{ws.max_row}"].value = f'=SUM(D{ws.max_row-len(ticker_gain_profit)}:D{ws.max_row-1})'
# ws[f"E{ws.max_row}"].value = f"=COUNTIF(E2:E{ws.max_row-1}, \"Gain\")/{len(All_Tickers)}"
# ws[f"F{ws.max_row}"].value = f"=COUNTIF(F2:F{ws.max_row-1}, \"Gain\")/{len(All_Tickers)}"

workbook.save(filename)

PCOR not found in active trade info
BRKB not found in active trade info
